In [1]:
# import moduel
import pandas as pd
import numpy as np
import os 
from os import listdir
import re

In [2]:
total_jongro = pd.DataFrame(columns=['미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
total_jongro 

,미세먼지(PM-10),소음(db),온도(℃),습도(%),초미세먼지(PM-2.5),PM10단계,PM25단계,HDONG_NM


In [3]:
# 종로1,2,3,4가동
# V10O1611173, V10O1611145
# V10O1610545, V10O1610567, V10O1612113, V10O1611750, V10O1611684, 
# V10O1610546, V10O1610540, V10O1610542, V10O1610543, V10O1610544,

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611173'))
temp2 = pd.read_csv('{}.csv'.format('V10O1611145'))
temp3 = pd.read_csv('{}.csv'.format('V10O1610545'))
temp4 = pd.read_csv('{}.csv'.format('V10O1610567'))
temp5 = pd.read_csv('{}.csv'.format('V10O1612113'))
temp6 = pd.read_csv('{}.csv'.format('V10O1611750'))
temp7 = pd.read_csv('{}.csv'.format('V10O1611684'))
temp8 = pd.read_csv('{}.csv'.format('V10O1610546'))
temp9 = pd.read_csv('{}.csv'.format('V10O1610540'))
temp10 = pd.read_csv('{}.csv'.format('V10O1610542'))
temp11 = pd.read_csv('{}.csv'.format('V10O1610543'))
temp12 = pd.read_csv('{}.csv'.format('V10O1610544'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)
df = df.append(temp2)
df = df.append(temp3)
df = df.append(temp4)
df = df.append(temp5)
df = df.append(temp6)
df = df.append(temp7)
df = df.append(temp8)
df = df.append(temp9)
df = df.append(temp10)
df = df.append(temp11)
df = df.append(temp12)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '종로1.2.3.4가동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '종로1.2.3.4가동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


8760


In [4]:
# 가회동
# V10O1611634, V10O1611698, V10O1611722

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611634'))
temp2 = pd.read_csv('{}.csv'.format('V10O1611698'))
temp3 = pd.read_csv('{}.csv'.format('V10O1611722'))


# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)
df = df.append(temp2)
df = df.append(temp3)


# 측정기 고유번호 변경
df['측정기 고유번호'] = '가회동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '가회동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [5]:
# 혜화동
# V10O1611623, V10O1611645

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611623'))
temp2 = pd.read_csv('{}.csv'.format('V10O1611645'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)
df = df.append(temp2)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '혜화동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '혜화동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [6]:
# 사직동
# V01o1610468, V10O1611172

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V01o1610468'))
temp2 = pd.read_csv('{}.csv'.format('V10O1611172'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)
df = df.append(temp2)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '사직동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '사직동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [7]:
# 천연동
# V10O1611289

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611289'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '천연동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '천연동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [8]:
# 교남동
# V10O1610252

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1610252'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '교남동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '교남동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [9]:
# 명동
# V10O1611887

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611887'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '명동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '명동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [10]:
# 종로5,6가동
# V10O1611639

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611639'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '종로5.6가동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '종로5.6가동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
print(final['HDONG_NM'].value_counts())
# total_jongro = total_jongro.append(final)

8760
종로5.6가동    8760
Name: HDONG_NM, dtype: int64


In [11]:
# final['HDONG_NM'][6000:]

In [12]:
# 이화동
# V10O1611658

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611658'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '이화동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '이화동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [13]:
# 창신 1동
# V10O1611151, V10O1611220

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611151'))
temp2 = pd.read_csv('{}.csv'.format('V10O1611220'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)
df = df.append(temp2)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '창신1동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '창신1동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [14]:
# 부암동
# V10O1611170

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611170'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '부암동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '부암동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [15]:
# 숭인2동
# V10O1612106

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1612106'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '숭인2동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '숭인2동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [16]:
# 창신3동
# V10O1611251

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611251'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '창신3동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '창신3동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [17]:
# 평창동
# V10O1611258

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611258'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '평창동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '평창동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [18]:
# 청운효자동
# V10O1611255

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\종로구_전처리"
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp = pd.read_csv('{}.csv'.format('V10O1611255'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '청운효자동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '청운효자동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_jongro= total_jongro.append(final)

8760


In [19]:
len(total_jongro)

122640

In [20]:
total_jongro.to_csv('total_jongro.csv',encoding='cp949')

In [21]:
8760 * 10

87600

In [22]:
113880 + 140160

254040

In [23]:
254040 + 87600

341640

In [24]:
341640 - 8760 * 3

315360